In [51]:
import pandas as pd

import sys
sys.path.insert(1, '../../scripts/')
from s3_support import *

# load data

In [52]:
q = "select id, amount, useragent, source from transactions where status='A' and date>='2020-01-01' and date <'2021-01-01'"
trans = redshift_query_read(q, schema='public')

In [70]:
len(trans[trans['useragent'].isna()]) / len(trans)

0.7843395851405496

In [53]:
def tag_device(u):
    token_identifiers = {
        'Mobile': ['iphone', 'android', 'mobilevt4', 'Givi2',
                   'Linuxarmv7', 'mobilevt100', 'Givi1'],
        'Tablet': ['ipad', 'Qgiv20Kiosk'],
        'Desktop': ['MacintoshIntelMacOSX', 'WindowsNT',
                    'CrOS', 'Linuxx8664', 'UbuntuLinuxi686',
                    'UbuntuLinuxi686', 'ColdFusion', 
                    'Linuxi686', 'MacintoshUPPC',
                    'BluebeamRevu', 'Mozilla50X11Linuxaarch64',
                    'Mozilla50X11Linuxrv'],
        'TV': ['SMARTTV']
    }
    
    for k in token_identifiers:
        for t in token_identifiers[k]:
            if t.lower() in str(u).lower():
                return k
    
    return None

In [54]:
trans['device'] = trans['useragent'].apply(tag_device)

In [55]:
print("{:.4f}".format(len(trans[(trans['device'].isna())&(~trans['useragent'].isna())]) / len(trans[~trans['useragent'].isna()])))
trans[(trans['device'].isna())&(~trans['useragent'].isna())].tail()

0.0000


,id,amount,useragent,source,device
1877443,10866351,103.0,jacapps,don_form,None
2051632,11152037,2.0,okhttp3100,don_form,None
2055826,11154100,2.0,okhttp3100,don_form,None
2060235,11152028,2.0,okhttp3100,don_form,None
2095980,11154078,2.0,okhttp3100,don_form,None


In [63]:
trans_all = trans.groupby('device')['id'].nunique().reset_index()
trans_all.columns = ['device', 'count']
trans_all['percentage'] = trans_all['count'] / trans_all['count'].sum(axis=0)
trans_all

,device,count,percentage
0,Desktop,313564,0.675218
1,Mobile,139536,0.300472
2,TV,4,0.000009
3,Tablet,11285,0.024301


In [67]:
trans_frontend = trans[~trans['source'].isin(['mobilevt', 'vt', 'kiosk'])].groupby('device')['id'].nunique().reset_index()
trans_frontend.columns = ['device', 'count']
trans_frontend['percentage'] = trans_frontend['count'] / trans_frontend['count'].sum(axis=0)
trans_frontend

,device,count,percentage
0,Desktop,297071,0.669208
1,Mobile,138401,0.311774
2,TV,4,0.000009
3,Tablet,8438,0.019008


In [68]:
trans_exsms = trans[~trans['source'].isin(['mobilevt', 'vt', 'kiosk', 'sms'])].groupby('device')['id'].nunique().reset_index()
trans_exsms.columns = ['device', 'count']
trans_exsms['percentage'] = trans_exsms['count'] / trans_exsms['count'].sum(axis=0)
trans_exsms

,device,count,percentage
0,Desktop,296972,0.677279
1,Mobile,133072,0.303486
2,TV,4,0.000009
3,Tablet,8430,0.019226
